**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](manipulation.csail.mit.edu/pose.html).**

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='20200915', drake_build='nightly')

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Setup rendering (with xvfb), if necessary:
import os
if 'google.colab' in sys.modules and os.getenv("DISPLAY") is None:
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(1400, 900))
    display.start()

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
server_args = []
if 'google.colab' in sys.modules:
    server_args = ['--ngrok_http_tunnel']
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

In [ ]:
# Let's do all of our imports here, too.
import numpy as np
import matplotlib.animation as animation
import matplotlib.pyplot as plt, mpld3
import pydot
from IPython.display import display, SVG, HTML
from sklearn.neighbors import NearestNeighbors

from pydrake.all import (
    AddMultibodyPlantSceneGraph, BaseField, Box, ConnectMeshcatVisualizer, DepthCameraProperties, 
    DepthImageToPointCloud, DiagramBuilder, FindResourceOrThrow, FixedOffsetFrame, 
    GeometryInstance, MakeRenderEngineVtk, MakePhongIllustrationProperties, 
    MeshcatPointCloudVisualizer, Parser, RandomGenerator, RenderEngineVtkParams, RgbdSensor, RigidTransform, 
    RollPitchYaw, RotationMatrix, SpatialInertia, set_log_level, UniformlyRandomRotationMatrix, world_model_instance)
from pydrake.examples.manipulation_station import ManipulationStation

if running_as_notebook:
    mpld3.enable_notebook()

# Simulating an RGB-D camera



In [ ]:
def DepthCameraDemoSystem():
    builder = DiagramBuilder()

    # Create the physics engine + scene graph.
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    # Add a single object into it.
    X_Mustard = RigidTransform(RollPitchYaw(-np.pi/2., 0, -np.pi/2.), [0, 0, 0.09515])
    mustard = Parser(plant).AddModelFromFile(FindResourceOrThrow(
        "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf"))
    plant.WeldFrames(plant.world_frame(), 
                     plant.GetFrameByName("base_link_mustard", mustard), 
                     X_Mustard)
    # Add a rendering engine
    renderer = "my_renderer"
    scene_graph.AddRenderer(renderer,
                            MakeRenderEngineVtk(RenderEngineVtkParams()))

    # TODO(russt): Use model directives for this
    # Add a box for the camera in the environment.
    X_Camera = RigidTransform(
        RollPitchYaw(0, -0.2, 0.2).ToRotationMatrix().multiply(
            RollPitchYaw(-np.pi/2.0, 0, np.pi/2.0).ToRotationMatrix()),
        [.5, .1, .2])
    camera_instance = plant.AddModelInstance("cameras")
    camera = plant.AddRigidBody("camera", camera_instance, SpatialInertia())
    plant.RegisterVisualGeometry(
        camera, RigidTransform([0, 0, -0.01]), Box(width=.1, depth=.02, height=.02), "D415", 
        [.4, .4, .4, 1.])
    plant.WeldFrames(plant.world_frame(), camera.body_frame(), X_Camera)

    plant.Finalize()

    frames_to_draw = {"cameras": "camera"}
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
                                       delete_prefix_on_load=False,
                                       frames_to_draw=frames_to_draw)
    meshcat.load()

    properties = DepthCameraProperties(width=640,
                                       height=480,
                                       fov_y=np.pi / 4.0,
                                       renderer_name=renderer,
                                       z_near=0.1,
                                       z_far=10.0)
    camera = builder.AddSystem(
        RgbdSensor(parent_id=plant.GetBodyFrameIdOrThrow(camera.index()),
                   X_PB=RigidTransform(),
                   properties=properties,
                   show_window=False))
    camera.set_name("rgbd_sensor")
    builder.Connect(scene_graph.get_query_output_port(),
                    camera.query_object_input_port())

    # Export the camera outputs
    builder.ExportOutput(camera.color_image_output_port(), "color_image")
    builder.ExportOutput(camera.depth_image_32F_output_port(), "depth_image")

    # Add a system to convert the camera output into a point cloud
    to_point_cloud = builder.AddSystem(
        DepthImageToPointCloud(camera_info=camera.depth_camera_info(),
                               fields=BaseField.kXYZs | BaseField.kRGBs))
    builder.Connect(camera.depth_image_32F_output_port(),
                    to_point_cloud.depth_image_input_port())
    builder.Connect(camera.color_image_output_port(),
                    to_point_cloud.color_image_input_port())

    # Send the point cloud to meshcat for visualization, too.
    meshcat_pointcloud = builder.AddSystem(MeshcatPointCloudVisualizer(meshcat, X_WP=X_Camera))
    builder.Connect(to_point_cloud.point_cloud_output_port(), meshcat_pointcloud.get_input_port())

    # Export the point cloud output.
    builder.ExportOutput(to_point_cloud.point_cloud_output_port(),
                         "point_cloud")

    diagram = builder.Build()
    diagram.set_name("depth_camera_demo_system")
    return diagram


In [ ]:
def plot_camera_images():
    system = DepthCameraDemoSystem()

    # Evaluate the camera output ports to get the images.
    context = system.CreateDefaultContext()
    system.Publish(context)
    color_image = system.GetOutputPort("color_image").Eval(context)
    depth_image = system.GetOutputPort("depth_image").Eval(context)

    # Plot the two images.
    plt.subplot(121)
    plt.imshow(color_image.data)
    plt.title('Color image')
    plt.subplot(122)
    plt.imshow(np.squeeze(depth_image.data))
    plt.title('Depth image')
    #mpld3.display()
    plt.show()

plot_camera_images()

In [ ]:
def draw_diagram():
    system = DepthCameraDemoSystem()
    display(SVG(pydot.graph_from_dot_data(system.GetGraphvizString(max_depth=1))[0].create_svg()))

draw_diagram()


In [ ]:
def plot_manipulation_station_camera_images():
    station = ManipulationStation()
    station.SetupManipulationClassStation()
    # station.SetupClutterClearingStation()
    station.Finalize()
    context = station.CreateDefaultContext()

    camera_names = station.get_camera_names()
    index = 1
    for name in camera_names:
        color_image = station.GetOutputPort("camera_" + name +
                                            "_rgb_image").Eval(context)
        depth_image = station.GetOutputPort("camera_" + name +
                                            "_depth_image").Eval(context)

        plt.subplot(len(camera_names), 2, index)
        plt.imshow(color_image.data)
        index += 1
        plt.title('Color image')
        plt.subplot(len(camera_names), 2, index)
        plt.imshow(np.squeeze(depth_image.data))
        index += 1
        plt.title('Depth image')

    plt.show()

plot_manipulation_station_camera_images()    

# Point cloud registration with known correspondences

In [ ]:
def MakeModelAndScenePoints(Nm = 20):
    """ Returns p_Om, p_s """
    random_state = np.random.RandomState()

    # Make a random set of points to define our object in the x,y plane
    l = 1.0 + 0.4*random_state.rand(1, Nm)
    theta = np.arange(0, 2.0*np.pi, 2.0*np.pi/Nm)
    p_Om = np.vstack((l * np.sin(theta), l * np.cos(theta), 0 * l))

    # Make a random object pose, and apply it to get the scene points.
    X_O = RigidTransform(RotationMatrix.MakeZRotation(np.pi*random_state.random()), 2.0*random_state.rand(3, 1))
    p_s = X_O.multiply(p_Om)  # Consider adding noise here.
    
    return p_Om, p_s

def PlotEstimate(p_Om, p_s, Xhat_O=RigidTransform(), chat=None, ax=None):
    p_m = Xhat_O.multiply(p_Om)
    if ax is None:
        ax = plt.subplot()
    artists = ax.plot(p_s[0, :], p_s[1, :], 'ro')
    artists += ax.fill(p_s[0, :], p_s[1, :], 'lightsalmon')
    artists += ax.plot(p_m[0, :], p_m[1, :], 'bo')
    artists += ax.fill(p_m[0, :], p_m[1, :], 'lightblue', alpha=0.5)
    if chat is not None:
        artists += ax.plot(np.vstack((p_m[0, chat], p_s[0, :])), np.vstack((p_m[1, chat], p_s[1, :])), 'g--')
    ax.axis('equal')
    return artists

def PoseEstimationGivenCorrespondences(p_Om, p_s, chat):
    """ Returns optimal X_O given the correspondences """
    # Apply correspondences, and transpose data to support numpy broadcasting
    p_Omc = p_Om[:, chat].T
    p_s = p_s.T

    # Calculate the central points
    p_Ombar = p_Omc.mean(axis=0)
    p_sbar = p_s.mean(axis=0)

    # Calculate the "error" terms, and form the data matrix
    merr = p_Omc - p_Ombar
    serr = p_s - p_sbar
    W = np.matmul(serr.T, merr)

    # Compute R
    U, Sigma, Vt = np.linalg.svd(W)
    R = np.matmul(U, Vt)
    if np.linalg.det(R) < 0:
       print("fixing improper rotation")
       Vt[m - 1, :] *= -1
       R = np.matmul(U, Vt)

    # Compute p
    p = p_sbar - np.matmul(R, p_Ombar)

    return RigidTransform(RotationMatrix(R), p)


Nm = 20
p_Om, p_s = MakeModelAndScenePoints(Nm=20)
Xhat = RigidTransform()
c = np.arange(0,Nm)  # perfect, known correspondences
fig, ax = plt.subplots(1, 2)
PlotEstimate(p_Om, p_s, Xhat, c, ax=ax[0])
Xhat = PoseEstimationGivenCorrespondences(p_Om, p_s, c)
ax[1].set_xlim(ax[0].get_xlim())
ax[1].set_ylim(ax[0].get_ylim());
PlotEstimate(p_Om, p_s, Xhat, c, ax=ax[1])
ax[0].set_title('Original Data')
ax[1].set_title('After Registration');

# Iterative Closest Point (ICP)

In [ ]:
def FindClosestPoints(point_cloud_A, point_cloud_B):
    """
    Finds the nearest (Euclidean) neighbor in point_cloud_B for each
    point in point_cloud_A.
    @param point_cloud_A A 3xN numpy array of points.
    @param point_cloud_B A 3xN numpy array of points.
    @return indices An (N, ) numpy array of the indices in point_cloud_B of each
        point_cloud_A point's nearest neighbor.
    @return distances An (N, ) numpy array of Euclidean distances from each
        point in point_cloud_A to its nearest neighbor in point_cloud_B.
    """
    distances = np.zeros(point_cloud_A.shape[1])
    indices = np.zeros(point_cloud_A.shape[1])

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(point_cloud_B.T)
    distances, indices = neigh.kneighbors(point_cloud_A.T, return_distance=True)

    distances = distances.ravel()
    indices = indices.ravel()

    return indices, distances

Nm = 20
p_Om, p_s = MakeModelAndScenePoints(Nm=20)
Xhat = RigidTransform()
chat_previous = np.zeros(Nm)

fig, ax = plt.subplots()
frames = []
frames.append(PlotEstimate(p_Om=p_Om, p_s=p_s, Xhat_O=Xhat, chat=None, ax=ax))

while True:
    chat, distances = FindClosestPoints(p_s, Xhat.multiply(p_Om))
    if np.array_equal(chat, chat_previous):
        # Then I've converged.
        break
    chat_previous = chat
    frames.append(PlotEstimate(p_Om=p_Om, p_s=p_s, Xhat_O=Xhat, chat=chat, ax=ax))
    Xhat = PoseEstimationGivenCorrespondences(p_Om, p_s, chat)
    frames.append(PlotEstimate(p_Om=p_Om, p_s=p_s, Xhat_O=Xhat, chat=None, ax=ax))

ani = animation.ArtistAnimation(fig, frames, interval=400)

display(HTML(ani.to_jshtml()))
plt.close()